Make imports relative

In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
home = str(Path.home()) # all other paths are relative to this path. change to something else if this is not the case on your system

Load data from CSV (ensure encoding is UTF-8 -- can be done by using save as in Excel)

In [2]:
import pandas as pd

df = pd.read_csv('./final_senate_tweets_utf8.csv', encoding='utf-8')
df

,Unnamed: 0,index,id,screen_name,user_id,time,link,text,source,tweet_id,...,tweet_object,full_name,full_text,followers_count,following_count,num_posts_user_liked,num_tweets_user_posted,is_quote,tweet_retweets,tweet_likes
0,0,2300,1.296210e+18,DainesforMT,2.147677e+08,2020-08-19T18:00:00-04:00,https://www.twitter.com/DainesforMT/statuses/1...,"Unlike my liberal opponent, I will always vote...",TweetDeck,1.296210e+18,...,Status(_api=<tweepy.api.API object at 0x000001...,Team Daines,"Unlike my liberal opponent, I will always vote...",3822,299,36,1791,False,0,2
1,1,860,1.242920e+18,SenatorDurbin,2.473346e+08,2020-03-25T17:22:44-04:00,https://www.twitter.com/SenatorDurbin/statuses...,"Beyond the Stateâ€™s ongoing response efforts,...",Twitter for iPhone,1.242920e+18,...,Status(_api=<tweepy.api.API object at 0x000001...,Senator Dick Durbin,"Beyond the Stateâ€™s ongoing response efforts,...",743839,2416,734,21314,False,32,135
2,2,247,1.221810e+18,HawleyMO,2.352629e+09,2020-01-27T09:45:36-05:00,https://www.twitter.com/HawleyMO/statuses/1221...,I say again: if the Senate is going to call wi...,Twitter for iPhone,1.221810e+18,...,Status(_api=<tweepy.api.API object at 0x000001...,Josh Hawley,I say again: if the Senate is going to call wi...,723535,955,605,5974,False,6146,23184
3,3,1315,1.288150e+18,SenSchumer,1.749401e+07,2020-07-28T12:20:05-04:00,https://www.twitter.com/SenSchumer/statuses/12...,We all want our schools to re-open.\n\nBut the...,Twitter Media Studio,1.288150e+18,...,Status(_api=<tweepy.api.API object at 0x000001...,Chuck Schumer,We all want our schools to re-open.\n\nBut the...,3389425,24249,993,22392,False,224,754
4,4,3106,1.294060e+18,SenJeffMerkley,2.920105e+07,2020-08-13T19:43:01-04:00,https://www.twitter.com/SenJeffMerkley/statuse...,@wweek confirmed that the Trump administration...,Twitter for iPhone,1.294060e+18,...,Status(_api=<tweepy.api.API object at 0x000001...,Senator Jeff Merkley,@wweek confirmed that the Trump administration...,511222,1075,2086,15265,False,584,825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8324,9995,1563,1.232680e+18,SenSchumer,1.749401e+07,2020-02-26T10:06:09-05:00,https://www.twitter.com/SenSchumer/statuses/12...,To the Department of Justice:\n\nRichard Grene...,Twitter Web App,1.232680e+18,...,Status(_api=<tweepy.api.API object at 0x000001...,Chuck Schumer,To the Department of Justice:\n\nRichard Grene...,3389778,24249,997,22396,False,2020,4693
8325,9996,1169,1.312410e+18,SenRickScott,1.315461e+08,2020-10-03T10:56:29-04:00,https://www.twitter.com/SenRickScott/statuses/...,I misspoke this morning in my @FoxNews intervi...,Twitter for iPhone,1.312410e+18,...,Status(_api=<tweepy.api.API object at 0x000001...,Rick Scott,I misspoke this morning in my @FoxNews intervi...,368034,1518,39,21172,False,1368,6980
8326,9997,2625,1.237420e+18,SenMarkey,2.140683e+07,2020-03-10T12:41:59-04:00,https://www.twitter.com/SenMarkey/statuses/123...,"During an emergency, whether itâ€™s a house fi...",Twitter Web App,1.237420e+18,...,Status(_api=<tweepy.api.API object at 0x000001...,Ed Markey,"During an emergency, whether itâ€™s a house fi...",251127,1777,1042,18535,False,6,26
8327,9998,910,1.253090e+18,SenFeinstein,4.762569e+08,2020-04-22T18:33:00-04:00,https://www.twitter.com/SenFeinstein/statuses/...,The next round of coronavirus relief legislati...,TweetDeck,1.253090e+18,...,Status(_api=<tweepy.api.API object at 0x000001...,Senator Dianne Feinstein,The next round of coronavirus relief legislati...,1500108,256,112,13174,False,26,123


Sentiment Analysis

Sources:
- https://www.analyticsvidhya.com/blog/2021/06/vader-for-sentiment-analysis/
- https://www.nltk.org/howto/sentiment.html
- https://www.digitalocean.com/community/tutorials/how-to-perform-sentiment-analysis-in-python-3-using-the-natural-language-toolkit-nltk

In [3]:
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer

def remove_links_and_handles(token):
    clean_token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
    clean_token = re.sub("(@[A-Za-z0-9_]+)","", clean_token)
    return clean_token

def clean_tweet(text):
    tokens = text.split()
    filtered_tokens = map(remove_links_and_handles, tokens)
    filtered_tokens = list(filter(lambda t: len(t) > 0, filtered_tokens))
    clean_text = " ".join(filtered_tokens)
    return clean_text

def analyze_sentiment(text):
    sid = SentimentIntensityAnalyzer()
    return sid.polarity_scores(clean_tweet(text))

df['is_rt'] = df['text'].str.startswith('RT')
# df['sentiment'] = df['text'].apply(analyze_sentiment)
df = df.dropna()
sentimentDf = pd.DataFrame(df['full_text'].apply(analyze_sentiment).tolist())
joinedSentimentDf = pd.concat([df, sentimentDf], axis=1)
joinedSentimentDf

,Unnamed: 0,index,id,screen_name,user_id,time,link,text,source,tweet_id,...,following_count,num_posts_user_liked,num_tweets_user_posted,is_quote,tweet_retweets,tweet_likes,neg,neu,pos,compound
0,0,2300,1.296210e+18,DainesforMT,2.147677e+08,2020-08-19T18:00:00-04:00,https://www.twitter.com/DainesforMT/statuses/1...,"Unlike my liberal opponent, I will always vote...",TweetDeck,1.296210e+18,...,299,36,1791,False,0,2,0.000,1.000,0.000,0.0000
1,1,860,1.242920e+18,SenatorDurbin,2.473346e+08,2020-03-25T17:22:44-04:00,https://www.twitter.com/SenatorDurbin/statuses...,"Beyond the Stateâ€™s ongoing response efforts,...",Twitter for iPhone,1.242920e+18,...,2416,734,21314,False,32,135,0.189,0.811,0.000,-0.7906
2,2,247,1.221810e+18,HawleyMO,2.352629e+09,2020-01-27T09:45:36-05:00,https://www.twitter.com/HawleyMO/statuses/1221...,I say again: if the Senate is going to call wi...,Twitter for iPhone,1.221810e+18,...,955,605,5974,False,6146,23184,0.000,1.000,0.000,0.0000
3,3,1315,1.288150e+18,SenSchumer,1.749401e+07,2020-07-28T12:20:05-04:00,https://www.twitter.com/SenSchumer/statuses/12...,We all want our schools to re-open.\n\nBut the...,Twitter Media Studio,1.288150e+18,...,24249,993,22392,False,224,754,0.000,0.827,0.173,0.6652
4,4,3106,1.294060e+18,SenJeffMerkley,2.920105e+07,2020-08-13T19:43:01-04:00,https://www.twitter.com/SenJeffMerkley/statuse...,@wweek confirmed that the Trump administration...,Twitter for iPhone,1.294060e+18,...,1075,2086,15265,False,584,825,0.219,0.686,0.094,-0.8122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8324,9995,1563,1.232680e+18,SenSchumer,1.749401e+07,2020-02-26T10:06:09-05:00,https://www.twitter.com/SenSchumer/statuses/12...,To the Department of Justice:\n\nRichard Grene...,Twitter Web App,1.232680e+18,...,24249,997,22396,False,2020,4693,0.190,0.692,0.118,-0.7506
8325,9996,1169,1.312410e+18,SenRickScott,1.315461e+08,2020-10-03T10:56:29-04:00,https://www.twitter.com/SenRickScott/statuses/...,I misspoke this morning in my @FoxNews intervi...,Twitter for iPhone,1.312410e+18,...,1518,39,21172,False,1368,6980,0.254,0.746,0.000,-0.6633
8326,9997,2625,1.237420e+18,SenMarkey,2.140683e+07,2020-03-10T12:41:59-04:00,https://www.twitter.com/SenMarkey/statuses/123...,"During an emergency, whether itâ€™s a house fi...",Twitter Web App,1.237420e+18,...,1777,1042,18535,False,6,26,0.294,0.606,0.100,-0.8807
8327,9998,910,1.253090e+18,SenFeinstein,4.762569e+08,2020-04-22T18:33:00-04:00,https://www.twitter.com/SenFeinstein/statuses/...,The next round of coronavirus relief legislati...,TweetDeck,1.253090e+18,...,256,112,13174,False,26,123,0.064,0.753,0.183,0.6369


In [4]:
noRTs = joinedSentimentDf[joinedSentimentDf.is_rt == False]
congress_groups = noRTs.groupby(['full_name', 'user_id', 'screen_name'], as_index=False).mean()
congress_groups['num_tweets_analyzed'] = noRTs.groupby(['full_name', 'user_id', 'screen_name'], as_index=False).count()['index']

# drop useless cols
congress_groups.drop(columns=['id', 'tweet_id', 'is_rt'], inplace=True)
congress_groups

,full_name,user_id,screen_name,Unnamed: 0,index,followers_count,following_count,num_posts_user_liked,num_tweets_user_posted,is_quote,tweet_retweets,tweet_likes,neg,neu,pos,compound,num_tweets_analyzed
0,Amy Klobuchar,3.353797e+07,amyklobuchar,4925.986207,1736.475862,1.885511e+06,133676.655172,18.0,14815.186207,False,1329.275862,8331.634483,0.071883,0.786117,0.142034,0.258987,145
1,Ben Cardin,2.780945e+08,BenCardinforMD,5168.000000,2328.800000,8.296000e+03,1735.000000,1665.0,2512.000000,False,2.000000,8.000000,0.171600,0.679400,0.149000,0.357600,5
2,Ben Ray LujÃ¡n,9.112531e+07,benraylujan,4638.257576,1550.530303,1.134003e+04,1055.000000,595.0,2440.000000,False,12.045455,33.666667,0.056788,0.785545,0.157621,0.340077,66
3,Bernie Sanders,2.944231e+07,SenSanders,4947.929577,1356.788732,1.245204e+07,2327.000000,365.0,23082.000000,False,3549.830986,19080.183099,0.144338,0.733535,0.122113,-0.122070,71
4,Bernie Sanders,2.167766e+08,BernieSanders,5324.156627,1442.228916,1.550551e+07,1472.000000,951.0,20509.000000,False,5415.915663,30416.548193,0.090801,0.776199,0.132970,0.138334,166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,Todd Young,5.996980e+07,ToddYoungIN,5036.000000,1876.458333,1.171910e+04,1204.437500,667.0,7416.229167,False,40.187500,50.708333,0.060479,0.731313,0.145687,0.256060,48
132,Tom Cotton,9.686504e+08,SenTomCotton,5207.803279,1647.639344,4.650590e+05,359.000000,122.0,11037.000000,False,779.245902,2297.081967,0.084377,0.775016,0.140705,0.171521,61
133,"U.S. Senator Bill Cassidy, M.D.",1.017500e+18,SenBillCassidy,4946.917526,1132.525773,4.555584e+04,1317.226804,481.0,5442.000000,False,11.113402,50.309278,0.058629,0.759835,0.181495,0.400510,97
134,U.S. Senator Cindy Hyde-Smith,9.833480e+17,SenHydeSmith,4452.200000,1236.085714,2.238649e+04,659.000000,1479.0,2660.628571,False,10.971429,35.971429,0.049657,0.751400,0.199029,0.491691,35


Code inspired by https://towardsdatascience.com/joining-dataframes-by-substring-match-with-python-pandas-8fcde5b03933

In [5]:
# Join congress groups with approval rating
approval = pd.read_csv('./ideology_approval.csv')
approval.drop([73], inplace=True)
approval['extremism'] = ((approval['ideology'] - approval['ideology'].mean()) / approval['ideology'].std()).abs()
congress_groups['join'] = 1
approval['join'] = 1
tweets_and_approval_full = congress_groups.merge(approval, on='join').drop('join', axis=1)
approval.drop('join', axis=1, inplace=True)
tweets_and_approval_full['match'] = tweets_and_approval_full.apply(lambda x: x.full_name.find(x.Name), axis=1).ge(0)
tweets_and_approval = tweets_and_approval_full[tweets_and_approval_full['match']].drop(columns=['match'])
tweets_and_approval

,full_name,user_id,screen_name,Unnamed: 0_x,index,followers_count,following_count,num_posts_user_liked,num_tweets_user_posted,is_quote,...,compound,num_tweets_analyzed,Unnamed: 0_y,ideology,last_name,Name,Positive,Neutral,Negative,extremism
8,Amy Klobuchar,3.353797e+07,amyklobuchar,4925.986207,1736.475862,1.885511e+06,133676.655172,18.0,14815.186207,False,...,0.258987,145,8,0.138044,Klobuchar,Amy Klobuchar,56%,14%,31%,1.384999
124,Ben Cardin,2.780945e+08,BenCardinforMD,5168.000000,2328.800000,8.296000e+03,1735.000000,1665.0,2512.000000,False,...,0.357600,5,26,0.275344,Cardin,Ben Cardin,49%,28%,23%,0.920336
295,Bernie Sanders,2.944231e+07,SenSanders,4947.929577,1356.788732,1.245204e+07,2327.000000,365.0,23082.000000,False,...,-0.122070,71,1,0.016945,Sanders,Bernie Sanders,65%,6%,29%,1.794833
393,Bernie Sanders,2.167766e+08,BernieSanders,5324.156627,1442.228916,1.550551e+07,1472.000000,951.0,20509.000000,False,...,0.138334,166,1,0.016945,Sanders,Bernie Sanders,65%,6%,29%,1.794833
556,"Bill Cassidy, M.D.",5.567743e+07,BillCassidy,5634.672414,1831.913793,4.686253e+04,1278.000000,171.0,7231.931034,False,...,0.358007,58,66,0.762907,Cassidy,Bill Cassidy,47%,27%,26%,0.729715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12811,Tim Scott,2.175432e+08,SenatorTimScott,5144.653061,1787.897959,6.136488e+05,3777.000000,3443.0,10675.000000,False,...,0.386318,49,71,0.823567,Scott,Tim Scott,50%,26%,24%,0.935006
12900,Todd Young,5.996980e+07,ToddYoungIN,5036.000000,1876.458333,1.171910e+04,1204.437500,667.0,7416.229167,False,...,0.256060,48,62,0.742281,Young,Todd Young,43%,34%,24%,0.659909
13027,Tom Cotton,9.686504e+08,SenTomCotton,5207.803279,1647.639344,4.650590e+05,359.000000,122.0,11037.000000,False,...,0.171521,61,92,0.904927,Cotton,Tom Cotton,48%,21%,31%,1.210349
13100,"U.S. Senator Bill Cassidy, M.D.",1.017500e+18,SenBillCassidy,4946.917526,1132.525773,4.555584e+04,1317.226804,481.0,5442.000000,False,...,0.400510,97,66,0.762907,Cassidy,Bill Cassidy,47%,27%,26%,0.729715


In [6]:
# Clean table
clean_tweets_approvals = tweets_and_approval.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y'])
clean_tweets_approvals.reset_index(inplace=True, drop=True)
clean_tweets_approvals = clean_tweets_approvals.astype({
    'followers_count': 'int',
    'num_posts_user_liked': 'int',
    'following_count': 'int',
    'num_tweets_user_posted': 'int'
})
clean_tweets_approvals['Positive'] = clean_tweets_approvals['Positive'].str.rstrip('%').astype('int')
clean_tweets_approvals['Neutral'] = clean_tweets_approvals['Neutral'].str.rstrip('%').astype('int')
clean_tweets_approvals['Negative'] = clean_tweets_approvals['Negative'].str.rstrip('%').astype('int')
clean_tweets_approvals.rename(columns={
    'full_name': 'twitter_full_name',
    'Name': 'name',
    'tweet_retweets': 'avg_retweets',
    'tweet_likes': 'avg_likes',
    'pos': 'avg_pos_sentiment', 
    'neu': 'avg_neutral_sentiment', 
    'neg': 'avg_neg_sentiment', 
    'compound': 'avg_compound_sentiment',
    'Positive': 'pos_approval',
    'Neutral': 'neutral_approval',
    'Negative': 'neg_approval',
}, inplace=True)
reorderedCols = ['name', 'screen_name', 'twitter_full_name', 'last_name', 'num_tweets_analyzed', 'user_id', 'following_count',
       'num_tweets_user_posted', 'followers_count', 'num_posts_user_liked',
       'avg_retweets', 'avg_likes', 'avg_neg_sentiment',
       'avg_neutral_sentiment', 'avg_pos_sentiment', 'avg_compound_sentiment',
       'ideology', 'extremism',  'pos_approval', 'neutral_approval',
       'neg_approval']
clean_tweets_approvals = clean_tweets_approvals[reorderedCols]

clean_tweets_approvals

,name,screen_name,twitter_full_name,last_name,num_tweets_analyzed,user_id,following_count,num_tweets_user_posted,followers_count,num_posts_user_liked,...,avg_likes,avg_neg_sentiment,avg_neutral_sentiment,avg_pos_sentiment,avg_compound_sentiment,ideology,extremism,pos_approval,neutral_approval,neg_approval
0,Amy Klobuchar,amyklobuchar,Amy Klobuchar,Klobuchar,145,3.353797e+07,133676,14815,1885511,18,...,8331.634483,0.071883,0.786117,0.142034,0.258987,0.138044,1.384999,56,14,31
1,Ben Cardin,BenCardinforMD,Ben Cardin,Cardin,5,2.780945e+08,1735,2512,8296,1665,...,8.000000,0.171600,0.679400,0.149000,0.357600,0.275344,0.920336,49,28,23
2,Bernie Sanders,SenSanders,Bernie Sanders,Sanders,71,2.944231e+07,2327,23082,12452037,365,...,19080.183099,0.144338,0.733535,0.122113,-0.122070,0.016945,1.794833,65,6,29
3,Bernie Sanders,BernieSanders,Bernie Sanders,Sanders,166,2.167766e+08,1472,20509,15505513,951,...,30416.548193,0.090801,0.776199,0.132970,0.138334,0.016945,1.794833,65,6,29
4,Bill Cassidy,BillCassidy,"Bill Cassidy, M.D.",Cassidy,58,5.567743e+07,1278,7231,46862,171,...,24.741379,0.069276,0.769638,0.161103,0.358007,0.762907,0.729715,47,27,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,Tim Scott,SenatorTimScott,Tim Scott,Scott,49,2.175432e+08,3777,10675,613648,3443,...,939.224490,0.047122,0.778082,0.174837,0.386318,0.823567,0.935006,50,26,24
118,Todd Young,ToddYoungIN,Todd Young,Young,48,5.996980e+07,1204,7416,11719,667,...,50.708333,0.060479,0.731313,0.145687,0.256060,0.742281,0.659909,43,34,24
119,Tom Cotton,SenTomCotton,Tom Cotton,Cotton,61,9.686504e+08,359,11037,465059,122,...,2297.081967,0.084377,0.775016,0.140705,0.171521,0.904927,1.210349,48,21,31
120,Bill Cassidy,SenBillCassidy,"U.S. Senator Bill Cassidy, M.D.",Cassidy,97,1.017500e+18,1317,5442,45555,481,...,50.309278,0.058629,0.759835,0.181495,0.400510,0.762907,0.729715,47,27,26


In [7]:
# Remove alt accounts
higher_followers = clean_tweets_approvals.groupby(['name']).max()
higher_followers = higher_followers['followers_count']

dropped = []
for i in clean_tweets_approvals.index:
    name = clean_tweets_approvals.iloc[i]['name']
    if clean_tweets_approvals.iloc[i]['followers_count'] < higher_followers[name]:
        dropped.append(i)
        
clean_tweet_no_alt = clean_tweets_approvals.drop(dropped)
clean_tweet_no_alt

,name,screen_name,twitter_full_name,last_name,num_tweets_analyzed,user_id,following_count,num_tweets_user_posted,followers_count,num_posts_user_liked,...,avg_likes,avg_neg_sentiment,avg_neutral_sentiment,avg_pos_sentiment,avg_compound_sentiment,ideology,extremism,pos_approval,neutral_approval,neg_approval
0,Amy Klobuchar,amyklobuchar,Amy Klobuchar,Klobuchar,145,3.353797e+07,133676,14815,1885511,18,...,8331.634483,0.071883,0.786117,0.142034,0.258987,0.138044,1.384999,56,14,31
3,Bernie Sanders,BernieSanders,Bernie Sanders,Sanders,166,2.167766e+08,1472,20509,15505513,951,...,30416.548193,0.090801,0.776199,0.132970,0.138334,0.016945,1.794833,65,6,29
4,Bill Cassidy,BillCassidy,"Bill Cassidy, M.D.",Cassidy,58,5.567743e+07,1278,7231,46862,171,...,24.741379,0.069276,0.769638,0.161103,0.358007,0.762907,0.729715,47,27,26
6,Brian Schatz,brianschatz,Brian Schatz,Schatz,126,4.774707e+07,3086,19537,389669,10077,...,5229.896825,0.081651,0.752857,0.149619,0.106227,0.223527,1.095699,48,23,29
7,Chris Murphy,ChrisMurphyCT,Chris Murphy,Murphy,119,1.500790e+08,233,26896,1076148,1370,...,4576.865546,0.091504,0.781034,0.119101,0.000751,0.201255,1.171074,47,19,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,Ted Cruz,tedcruz,Ted Cruz,Cruz,83,2.302269e+07,5377,35823,4754971,1252,...,5676.481928,0.085181,0.764120,0.126578,0.110524,0.913741,1.240179,49,17,34
115,Tim Kaine,timkaine,Tim Kaine,Kaine,29,1.728588e+08,772,11451,1000843,927,...,1692.862069,0.107241,0.753621,0.139034,0.027759,0.255174,0.988598,45,22,34
117,Tim Scott,SenatorTimScott,Tim Scott,Scott,49,2.175432e+08,3777,10675,613648,3443,...,939.224490,0.047122,0.778082,0.174837,0.386318,0.823567,0.935006,50,26,24
119,Tom Cotton,SenTomCotton,Tom Cotton,Cotton,61,9.686504e+08,359,11037,465059,122,...,2297.081967,0.084377,0.775016,0.140705,0.171521,0.904927,1.210349,48,21,31


Export to CSV

In [8]:
clean_tweet_no_alt.to_csv("final_no_alt_senate_tweets.csv")